## Random network distillation

In [4]:
#@title imports
import os
import time

from rl_trainer import RL_Trainer
from dqn_agent import DQNAgent
from core.dqn_utils import get_env_kwargs, PiecewiseSchedule, ConstantSchedule

%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'gym'

In [2]:
#@title runtime arguments

class Args:

  def __getitem__(self, key):
    return getattr(self, key)

  def __setitem__(self, key, val):
    setattr(self, key, val)

  def __contains__(self, key):
    return hasattr(self, key)

  env_name = 'PointmassHard-v0' 
  #env_name = 'PointmassEasy-v0' #@param ('PointmassEasy-v0', 'PointmassMedium-v0', 'PointmassHard-v0')
  exp_name = 'day5' #@param
  num_timesteps = 40000
  
  batch_size = 256 #@param {type: "integer"}

  num_agent_train_steps_per_iter = 1 #@param {type: "integer"}
  
  #@markdown Q-learning parameters
  double_q = True #@param {type: "boolean"}
  n_step = 1
    
  use_rnd = False
  num_exploration_steps = 20000
  
  rnd_output_size = 5
  rnd_n_layers = 2
  rnd_size = 400

  #@markdown system
  save_params = False #@param {type: "boolean"}
  no_gpu = False #@param {type: "boolean"}
  which_gpu = 0 #@param {type: "integer"}
  seed = 1 #@param {type: "integer"}

  #@markdown logging
  ## default is to not log video so
  ## that logs are small enough to be
  ## uploaded to gradscope
  video_log_freq =  -1 #@param {type: "integer"}
  scalar_log_freq =  5000#@param {type: "integer"}

  exploit_weight_schedule = ConstantSchedule(1.0)
  if use_rnd:
    explore_weight_schedule= PiecewiseSchedule([(0,1), (num_exploration_steps, 0)], outside_value=0.0)
  else:
    explore_weight_schedule= ConstantSchedule(0.0)
    
  if env_name=='PointmassEasy-v0':
    num_timesteps=10000
    num_exploration_steps=5000
  if env_name =='PointmassMedium-v0':
    num_timesteps=20000
    num_exploration_steps= 10000
  if env_name=='PointmassSpiral-v0':
    num_timesteps=30000
    num_exploration_steps=15000
  
args = Args()

args['train_batch_size'] = args['batch_size']

In [4]:
#@title create directories for logging

data_path = './data'

if not (os.path.exists(data_path)):
    os.makedirs(data_path)

logdir = args.exp_name + '_' + args.env_name[:-3]
if args['use_rnd']:
    logdir += '_rnd'
logdir = os.path.join(data_path, logdir)
args['logdir'] = logdir
if not(os.path.exists(logdir)):
    os.makedirs(logdir)

print("LOGGING TO: ", logdir)

LOGGING TO:  ./data/day5_PointmassEasy_rnd


In [5]:
#@title Define Q-function trainer

class Q_Trainer(object):

    def __init__(self, params):
        self.params = params
        train_args = {
            'num_agent_train_steps_per_iter': params['num_agent_train_steps_per_iter'],
            'train_batch_size': params['batch_size'],
            'double_q': params['double_q'],
        }

        env_args = get_env_kwargs(params['env_name'])
        env_args['num_timesteps'] = params['num_timesteps']

        for k, v in env_args.items():
          params[k] = v

        self.params['agent_class'] = DQNAgent
        self.params['agent_params'] = params
        self.params['train_batch_size'] = params['batch_size']
        self.params['env_wrappers'] = env_args['env_wrappers']

        self.rl_trainer = RL_Trainer(self.params)

    def run_training_loop(self):
        self.rl_trainer.run_training_loop(
            self.params['num_timesteps'],
            collect_policy = self.rl_trainer.agent.dqn,
            eval_policy = self.rl_trainer.agent.dqn,
            )

In [6]:
#@title run training

trainer = Q_Trainer(args)
trainer.run_training_loop()

GPU not detected. Defaulting to CPU.


/Users/air/anaconda3/envs/sds/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


ob_dim = Box(0.0, 10.0, (2,), float32), ac_dim = 5


  0%|          | 0/10000 [00:00<?, ?it/s]


********** Iteration 0 ************

Training agent...

Beginning logging procedure...
Timestep 1
Num Episodes 0
mean reward (100 episodes) nan
best mean reward -inf
running time 0.047187
	Train_EnvstepsSoFar : 1
	Train_EpisodeSoFar : 0
	Num_Episode_reach_the_goal : 0
	TimeSinceStart : 0.047187089920043945
Done logging...




  6%|▋         | 628/10000 [00:10<02:35, 60.11it/s] 


KeyboardInterrupt: 

In [7]:
import ipywidgets as widgets
import IPython.display as display


img1 = open(logdir+"/expl_last_traj.png", 'rb').read()
img2 = open(logdir+"/curr_state_density.png", 'rb').read()

wi1 = widgets.Image(value=img1, format='png', width=300, height=400)
wi2 = widgets.Image(value=img2, format='png', width=300, height=400)
sidebyside = widgets.HBox([wi1, wi2])
display.display(sidebyside)

FileNotFoundError: [Errno 2] No such file or directory: './data/day5_PointmassEasy_rnd/curr_state_density.png'